In [1]:
import tensorflow as tf

In [16]:
in_1=1
in_2=2
name=None #foo
# name='foo'

- tf.variable_scope使用default_name时，重名时顶层名称自动改变(后缀加上下划线和重复次数)

In [17]:
for idx in range(2):
#     with tf.variable_scope(name, 'V0', [in_1, in_2],reuse=tf.AUTO_REUSE):
    with tf.variable_scope(name, 'V0', [in_1, in_2]):
        with tf.variable_scope("bar"):
            v1 = tf.get_variable("var1", [1])   #变量空间需要设置reuse，否则第二次调用时报错
            v2 = tf.Variable(initial_value=[2.], name='var2', dtype=tf.float32)
            v3 = tf.Variable(initial_value=[2.], name='var2', dtype=tf.float32)
    with tf.Session() as sess:
        print('##ID: {}##'.format(idx))
        print ('v1 name: {}'.format(v1.name))
        print ('v2 name: {}'.format(v2.name))
        print ('v3 name: {}'.format(v3.name))

##ID: 0##
v1 name: V0_2/bar/var1:0
v2 name: V0_2/bar/var2:0
v3 name: V0_2/bar/var2_1:0
##ID: 1##
v1 name: V0_3/bar/var1:0
v2 name: V0_3/bar/var2:0
v3 name: V0_3/bar/var2_1:0


In [18]:
for idx in range(2):
    with tf.name_scope(name, 'V1', [in_1, in_2]):  #name和default_name都是依次递增_ID，全局唯一
        with tf.name_scope("bar"):
            #tf.get_variable_scope().reuse_variables()
            with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
                v1 = tf.get_variable("var1", [1])    
            v2 = tf.Variable(initial_value=[2.], name='var2', dtype=tf.float32)
            v3 = tf.Variable(initial_value=[2.], name='var2', dtype=tf.float32)
    
    with tf.Session() as sess:
        print('##ID: {}##'.format(idx))
        print('v1 name: {}'.format(v1.name))
        print('v2 name: {}'.format(v2.name))
        print('v3 name: {}'.format(v3.name))

##ID: 0##
v1 name: var1:0
v2 name: V1/bar/var2:0
v3 name: V1/bar/var2_1:0
##ID: 1##
v1 name: var1:0
v2 name: V1_1/bar/var2:0
v3 name: V1_1/bar/var2_1:0


- 使用reuse_variables函数时，tf.get_variable从已有变量中直接读取，否则报错；
- tf.get_variable的变量名取决于tf.variable_scope和自身的name值，与tf.name_scope无关；
- tf.Variable的变量名在同一空间重复时，名称自动改变(后缀加上下划线和重复次数)
- tf.Variable的变量空间重名时，名称自动改变(后缀加上下划线和重复次数)

In [3]:
v1 = tf.get_variable('foo/var1', [1])
for idx in range(2):
    with tf.variable_scope(name, 'V2', [in_1, in_2]) as scope:
        with tf.name_scope("bar"):
            scope.reuse_variables()  #仅当变量名已经存在时有效
            #tf.get_variable_scope().reuse_variables() #仅当变量名存在时有效
            v1 = tf.get_variable("var1", [1])
            v2 = tf.Variable(initial_value=[2.], name='var2', dtype=tf.float32)
            v3 = tf.Variable(initial_value=[2.], name='var2', dtype=tf.float32)

    with tf.Session() as sess:
        print('##ID: {}##'.format(idx))
        print('v1 name: {}'.format(v1.name))
        print('v2 name: {}'.format(v2.name))
        print('v3 name: {}'.format(v3.name))

Instructions for updating:
Colocations handled automatically by placer.
##ID: 0##
v1 name: foo/var1:0
v2 name: foo/bar/var2:0
v3 name: foo/bar/var2_1:0
##ID: 1##
v1 name: foo/var1:0
v2 name: foo_1/bar/var2:0
v3 name: foo_1/bar/var2_1:0


- tf.get_variable直接创建变量

In [4]:
v1 = tf.get_variable("foo_6/bar/var2", [1])
with tf.Session() as sess:
    print('v1 name: {}'.format(v1.name))

v1 name: foo_6/bar/var2:0


- tf.get_variable变量重名时需要指定reuse=True或tf.AUTO_REUSE，
- reuse=True或tf.AUTO_REUSE的区别时后者当变量不存在时可以创建，前者则不可以；两者均需要指定name，而不能使用default_name

In [5]:
v1 = tf.get_variable('foo_7', [1])
with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
    v1 = tf.get_variable('foo_7', [1])